In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
import optuna
from omegaconf import OmegaConf

from proT.euler_optuna.optuna_opt import OptunaStudy

def generate_summary_if_missing(folder_path: str):
    """Generate best_trial.yaml if missing by calling OptunaStudy.summary()"""
    
    if os.path.exists(join(folder_path, "best_trial.yaml")):
        return  # Already exists
    
    study_db = join(folder_path, "optuna", "study.db")
    if not os.path.exists(study_db):
        return  # No study to summarize
    
    try:
        # Get study name
        storage = f"sqlite:///{study_db}?timeout=60"
        study_name = optuna.get_all_study_summaries(storage=storage)[0].study_name
        
        # Create OptunaStudy and generate summary
        optuna_study = OptunaStudy(
            exp_dir=folder_path,
            data_dir="../data/input",
            cluster=False,
            study_name=study_name,
            manifest_tag="NA",
            study_path=join(folder_path, "optuna")
        )
        optuna_study.summary()
    except Exception as e:
        print(f"Failed to generate summary for {os.path.basename(folder_path)}: {e}")


c:\Users\ScipioneFrancesco\Documents\Projects\proT\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
baseline_result_path = "../experiments/baseline_optuna/euler/"
models = ["GRU", "LSTM", "proT", "proTNoNAIM", "TCN", "MLP"]
fusions = ["cat", "sum"]
datasets = ["ishigami", "dyconex"]

model_list  = []
fusion_list = []
dataset_list = []
trial_list = []
trainable_params = []
av_epoch_training_time = []
best_test_mae = []
best_test_r2 = []
df = None

def get_info_from_folder(folder: str, options: list, option_list: list):
    for opt in options:
        opt_str = "_" + opt + "_"
        if opt_str in folder:
            option_list.append(opt)
            

for folder in os.listdir(baseline_result_path):
    
    # Auto-generate summary if missing
    generate_summary_if_missing(folder_path=join(baseline_result_path, folder))
    
    get_info_from_folder(folder, models, model_list)
    get_info_from_folder(folder, fusions, fusion_list)
    get_info_from_folder(folder, datasets, dataset_list)
    
    config_path = join(baseline_result_path, folder, "best_trial.yaml")
    
    if not os.path.exists(config_path):
        print(f"Skipping {folder}: best_trial.yaml not available")
        continue
    
    config = OmegaConf.load(config_path)
    
    best_trial = config.trial_number
    best_trial_path = join(baseline_result_path, folder,"optuna",f"run_{best_trial}", "kfold_summary.json")
    
    k_fold_summary = OmegaConf.load(best_trial_path)
    av_epoch_training_time.append(k_fold_summary.best_fold.metrics.avg_time_per_epoch)
    trainable_params.append(k_fold_summary.best_fold.metrics.trainable_params)
    best_test_mae.append(k_fold_summary.best_fold.metrics.test_mae.split('(')[1].rstrip(')'))
    best_test_r2.append(k_fold_summary.best_fold.metrics.test_r2.split('(')[1].rstrip(')'))
    trial_list.append(best_trial_path)
    
    
    metrics = pd.DataFrame().from_dict(config.metrics, orient="index")
    
    if df is None:
        df = metrics
    else:
        df = pd.concat([df,metrics],ignore_index=True,axis=1)
    
df = df.transpose()
df["model"] = model_list
df["fusion"] = fusion_list
df["dataset"] = dataset_list
df["average_epoch_training_time"] = av_epoch_training_time
df["number_trainable_params"] = trainable_params
df["best fold test MAE"] = best_test_mae
df["best fold test R2"] = best_test_r2
df["best_trial"] = trial_list
round_digit = 3
df["test MAE"] = df["test_mae_mean"].round(round_digit).astype(str) + r"+/-" +df["test_mae_std"].round(round_digit).astype(str)
df["test R2"] = df["test_r2_mean"].round(round_digit).astype(str) + r"+/-" +df["test_r2_std"].round(round_digit).astype(str)

# print df
df = df[["model", "dataset", "test MAE","test R2", "best fold test MAE","best fold test R2", "average_epoch_training_time", "number_trainable_params", "best_trial"]]


filepath = "./output"
os.makedirs(filepath, exist_ok=True)
df.to_csv(join(filepath, "baseline_results.csv"))

df


Best Trial Summary
Trial number: 23
Optimization metric (val_loss): 0.079146

Best Parameters:
  d_hidden_set: 192
  n_layers_set: 3
  lr: 0.0006882470558446594
  dropout: 0.015310536021433695

Summary saved to: ..\experiments\baseline_optuna\euler\baseline_GRU_dyconex_sum_50590459\best_trial.yaml
Config path: /cluster/scratch/fscipion/baseline_GRU_dyconex_sum_50590459/optuna/run_23/config.yaml

Skipping _old: best_trial.yaml not available


,model,dataset,test MAE,test R2,best fold test MAE,best fold test R2,average_epoch_training_time,number_trainable_params,best_trial
0,GRU,dyconex,0.081+/-0.004,0.505+/-0.044,0.0758,0.5596,4.528631,654793,../experiments/baseline_optuna/euler/baseline_...
1,GRU,ishigami,0.037+/-0.002,0.426+/-0.006,0.0348,0.4280,2.628296,35137,../experiments/baseline_optuna/euler/baseline_...
2,LSTM,dyconex,0.651+/-0.631,-36.17+/-50.446,0.0819,0.5084,5.151879,340105,../experiments/baseline_optuna/euler/baseline_...
3,LSTM,ishigami,0.037+/-0.001,0.426+/-0.003,0.0360,0.4216,2.977574,370049,../experiments/baseline_optuna/euler/baseline_...
4,MLP,dyconex,0.085+/-0.015,0.484+/-0.14,0.0727,0.5823,0.827373,617737,../experiments/baseline_optuna/euler/baseline_...
5,MLP,ishigami,0.041+/-0.001,0.252+/-0.008,0.0396,0.2478,2.898114,580737,../experiments/baseline_optuna/euler/baseline_...
6,proTNoNAIM,dyconex,0.065+/-0.002,0.642+/-0.016,0.0648,0.6589,2.937452,1050301,../experiments/baseline_optuna/euler/baseline_...
7,proTNoNAIM,ishigami,0.003+/-0.0,0.998+/-0.0,0.0024,0.9987,7.474364,1164401,../experiments/baseline_optuna/euler/baseline_...
8,proT,dyconex,0.067+/-0.003,0.631+/-0.018,0.0664,0.6405,2.298916,968501,../experiments/baseline_optuna/euler/baseline_...
9,proT,ishigami,0.003+/-0.0,0.998+/-0.001,0.0024,0.9987,6.815514,1334701,../experiments/baseline_optuna/euler/baseline_...
